In [1]:
import json
from openai import OpenAI

In [2]:
from datasets import load_dataset
rag_dataset = load_dataset("neural-bridge/rag-dataset-1200")

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|█| 96
Generating test split: 100%|█| 240


In [3]:
type(rag_dataset)

datasets.dataset_dict.DatasetDict

In [4]:
rag_dataset['train']

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 960
})

In [5]:
rag_dataset['train'][1]['question']

'What are some of the potential negative impacts of charity as discussed in the context?'

In [6]:
rag_dataset['train'][1]['answer']

'The context discusses that charity can sometimes exacerbate problems rather than solve them. It can delay permanent infrastructure and job development, and in some cases, it can lead to mismanagement of funds. The context also mentions the situation in Haiti where an influx of donated rice and solar panels disrupted local businesses, making the local economy dependent on foreign aid. This is described as unsustainable and unethical. Furthermore, charity can also allow companies to overlook problems in their supply chain or company culture, as they can claim responsibility through their charitable actions.'

In [7]:
rag_dataset['train'][1]['context'].split("\n")

['A few nights ago, my husband and I were watching an episode of Dragon\'s Den, the Canadian version of Shark Tank, and a "social good" company was pitching their Haitian recycling program. It was a bad business concept poorly executed, but it didn\'t justify one of the Dragons\' responses, paraphrased below:',
 "You're going about this the wrong way! You don't create a social good business before you become profitable. You become profitable by whatever means necessary, and then you give back! That's how business works.",
 'Now, I\'ll concede that there\'s something to the idea that a business needs to be successful in order for a positive end to be achieved. But her response is not an unusual one in the business community, and more often than not, the American Sharks who invest in social enterprises also buy this "the ends justifies the means" argument. Put another way, business people as a rule - and no thanks to Capitalism - prioritize profit over social good. If some money can be t

In [8]:
instances_list = []

# Iterate over each instance in the 'train' split
for instance in rag_dataset['train']:
    # Create a dictionary for the current instance and append it to the list
    instances_list.append({
        "context": instance["context"],
        "question": instance["question"],
        "answer": instance["answer"]
    })

# Save the list of instances to a JSON file
with open('../data/documents.json', 'w', encoding='utf-8') as f:
    json.dump(instances_list, f, ensure_ascii=False, indent=4)

In [9]:
with open('../data/documents.json', 'rt') as f_in:
    documents = json.load(f_in)

In [10]:
documents[0]['question']

'Who found the answer to a search query collar george herbert essay?'

In [25]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [33]:
from elasticsearch import Elasticsearch

In [34]:

es_client = Elasticsearch('http://localhost:9200') 

In [35]:

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "context": {"type": "text"},
            "answer": {"type": "text"},
            "question": {"type": "text"}
        }
    }
}


In [37]:
index_name = "questions-related"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'questions-related'})

In [38]:
from tqdm.auto import tqdm

In [39]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████| 960/960 [00:26<00:00, 36.03it/s]


In [54]:
def elastic_search(query):
    # search_query = {
    #     "size": 5,
    #     "query": {
    #         "bool": {
    #             "must": {
    #                 "multi_match": {
    #                     "query": query,
    #                     "fields": ["question^3", "context^3", "answer"],
    #                     "type": "best_fields"
    #                 }
    #             }
    #         }
    #     }
    # }
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "context", "answer"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [55]:
query = 'How many percentage of the U.S. retail lingerie market did L-Brands account for in 2024?'

In [56]:
result_docs = elastic_search(query)

In [58]:
for i in range(5):
    
    print(result_docs[i]['answer'])

In 2016, L-Brands, the parent company of Victoria’s Secret, accounted for 83 percent of the U.S. retail lingerie market.
The person paid Rs 19,000 for the bicycle they purchased from Jackson Market.
The person ran 8 miles on Monday.
The parliament met for 135 times during the second session of the Legislature-Parliament.
The Twitter account @0109mahahu started being active since August 2013.


In [59]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "context^3", "answer"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    # search_query = {
    #     "size": 5,
    #     "query": {
    #         "bool": {
    #             "must": {
    #                 "multi_match": {
    #                     "query": query,
    #                     "fields": ["question^3", "context", "answer"],
    #                     "type": "best_fields"
    #                 }
    #             }
    #         }
    #     }
    # }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [60]:
result_docs = elastic_search(query)

In [62]:
for i in range(5):
    print(result_docs[i]['question'])
    print(result_docs[i]['answer'])
    print("-----------------------")

What percentage of the U.S. retail lingerie market did L-Brands, the parent company of Victoria’s Secret, account for in 2016?
In 2016, L-Brands, the parent company of Victoria’s Secret, accounted for 83 percent of the U.S. retail lingerie market.
-----------------------
What is the usual adult dosage for L-Methyl-MC NAC tablets?
The usual adult dose is one tablet daily or as prescribed by a licensed medical practitioner.
-----------------------
What are the terms of the new five-year pact between Verizon and the NFL?
Under the new five-year pact, Verizon will pay the NFL more than $2 billion to gain rights to live-stream all in-market and national games, including preseason, regular season, and playoff games — including the Super Bowl — to any mobile device nationwide. The deal also covers the NFL’s Thursday, Sunday and Monday Night Football games, and the Sunday day games that air on TV in viewers’ local market. Verizon’s rights extend to mobile phones across all carriers and across 